# TV Script Generation

Seinfeld : 미국 시트콤

이번 프로젝트에서는 Seinfeld라는 미국시트콤의 시즌 9의 대사(Script)를 데이터로 하여서 

그와 유사한 대사를 생성하는 모델을 만들 것 이다

## 00. Settings

In [1]:
import os
os.getcwd()

'C:\\Users\\USER\\Desktop\\Udacity\\TVScript_Generation\\code'

In [22]:
import numpy as np

---

## 01. Get the Data

Seinfeld script는 `\TVScript_Generation\data` 폴더내에 있다

<span style="color:red">[!]</span> 경로에서 '.'은 현재 디렉토리를 의미하고, '..'는 상위 디렉토리를 의미한다

따라서 `./../` 은 현재 디렉토리에서의 상위디렉토리, 즉 TVScript_Generation 폴더를 의미한다

<span style="color:red">[!]</span> Udacity에서 다운로드한 Seinfeld_Scripts.txt를 열라고하니 cpc949 문제가 일어났다

따라서 직접 메모장으로 ANSI로 변환 저장하여 Seinfeld_Scripts_1.txt로 오픈함

In [14]:
# helper.py의 load_data 함수를 이용하여 데이터를 불러오자

import helper

def load_data(path):
    """
    Load Dataset from File
    """
    input_file = os.path.join(path)
    with open(input_file, "r", encoding='utf-8') as f:
        data = f.read()

    return data

In [19]:
data_dir = './../data/Seinfeld_Scripts_1.txt'
text = helper.load_data(data_dir)

In [20]:
len(text)

3471464

In [105]:
type(text)

str

---

## 02. Explore the Data

Seinfeld 스크립트 텍스트 데이터는 모두 소문자이고, 각 대화는 `\n`으로 구분되어 있다.

이와 같은 특성들을 데이터를 직접보면서 확인할 수 있어야 한다.

In [36]:
print(f"텍스트데이터 길이 : {len(text)}")
print(f"The number of unique word : {len(set([word for word in text.split()]))}")
lines = text.split('\n')
print(f"The number of Lines : {len(lines)}")

텍스트데이터 길이 : 3471464
The number of unique word : 46367
The number of Lines : 109233


In [39]:
# 대사의 라인 살펴보기

print('\n'.join(text.split('\n')[10:30]))

jerry: oh, you dont recall? 

george: (on an imaginary microphone) uh, no, not at this time. 

jerry: well, senator, id just like to know, what you knew and when you knew it. 

claire: mr. seinfeld. mr. costanza. 

george: are, are you sure this is decaf? wheres the orange indicator? 

claire: its missing, i have to do it in my head decaf left, regular right, decaf left, regular right...its very challenging work. 

jerry: can you relax, its a cup of coffee. claire is a professional waitress. 

claire: trust me george. no one has any interest in seeing you on caffeine. 

george: how come youre not doing the second show tomorrow? 

jerry: well, theres this uh, woman might be coming in. 



---

## 03. Pre-processing Functions

### 3-1. Lookup Table

* input : `text`; words로 나눠진 TV scripts text data
* return : A tuple of dictionaries (`vocab_to_int`, `int_to_vocab`)

In [40]:
import collections
from collections import Counter

In [43]:
def create_lookup_tables(text):
    counts = Counter(text)
    vocab = sorted(counts, key=counts.get, reverse=True)
    
    vocab_to_int = {word: ii for ii, word in enumerate(vocab)}
    int_to_vocab = {ii: word for word, ii in vocab_to_int.items()}
    
    return (vocab_to_int, int_to_vocab)

In [45]:
# Udacity test function을 통한 create_lookup_tables함수 검증

import problem_unittests as tests
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### 3-2. Tokenize Punctuation

<span style="color:red">[!]</span> 구두점 : Token 로 이루어진 Dictionary를 만들어서 추후에 각 구두점(기호)을 하나의 단어처럼 만들고, 공백을 통해 구분할 수 있게끔 할 것임

<span style="color:skyblue">ex)</span> {'!' : '||Exclamation_mark||'}

In [47]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    구두점 : Token으로 이루어진 Dictionary 반환
    """
        
    return {
        '.' : '||Period||',
        ',' : '||Comma||',
        '"' : '||Quotation_Mark||',
        ';' : '||Semicolon||',
        '!' : '||Exclamation_mark||',
        '?' : '||Question_mark||',
        '(' : '||Left_Parentheses||',
        ')' : '||Right_Parentheses||',
        '-' : '||Dash||',
        '\n' : '||Return||'
    }

In [48]:
tests.test_tokenize(token_lookup)

Tests Passed


---

## 04. Pre-process the data and save it

In [50]:
# helper.py의 preprocess_and_save_data 함수를 통해서 
# 전처리된 텍스트데이터를 pickle로 저장하자

helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

In [51]:
SPECIAL_WORDS = {'PADDING': '<PAD>'}

def preprocess_and_save_data(dataset_path, token_lookup, create_lookup_tables):
    """
    Preprocess Text Data
    """
    text = load_data(dataset_path)
    
    # Ignore notice, since we don't use it for analysing the data
    # 앞부분은 버리네...?
    text = text[81:]

    # token_dict에 앞서만든 구두점 딕셔너리 할당
    # for loop로 text데이터의 구두점들을 모두 Token으로 replace
    token_dict = token_lookup()          
    for key, token in token_dict.items():
        text = text.replace(key, ' {} '.format(token))

    
    # 모두 소문자로 변경
    text = text.lower()
    
    # 단어 단위로 split
    text = text.split()

    vocab_to_int, int_to_vocab = create_lookup_tables(text + list(SPECIAL_WORDS.values()))
    int_text = [vocab_to_int[word] for word in text]
    pickle.dump((int_text, vocab_to_int, int_to_vocab, token_dict), open('preprocess.p', 'wb'))


* `dataset_path`를 받아서 앞부분은 버리네?
* 만들어두었던 구두점 토큰화 딕셔너리를 통해서 텍스트데이터내의 구두점들을 모두 ||Punctuation|| 이렇게 바꿈
* 소문자로 변경, 그리고 단어 단위로 쪼개기
* (패딩 Token도 추가해서) 단어<->정수 딕셔너리x2 만들기
* `int_text`에 단어-> 정수 변환시켜서 담아넣기
---
* pickle로 아래의 것들 저장
   * `int_text` : 단어, 구두점 -> 정수화 된 리스트
   * `vocab_to_int` : 단어->정수 딕셔너리
   * `int_to_vocab` : 정수->단어 딕셔너리
   * `token_dict` : 구두점->Token 딕셔너리

In [54]:
# 저장된 pickle파일 (preprocess.p) 불러와보기
import pickle

with open('preprocess.p', 'rb') as file:
    preprocessed_data = pickle.load(file)

In [58]:
int_text = preprocessed_data[0]
vocab_to_int = preprocessed_data[1]
int_to_vocab = preprocessed_data[2]
token_dict = preprocessed_data[3]

In [60]:
# 정수로 변환된 단어 토큰리스트
int_text[:10]

[24, 22, 47, 1, 1, 1, 17, 47, 22, 82]

In [64]:
# 단어 -> 정수 딕셔너리
vocab_to_int["||period||"]

1

In [65]:
# 정수 -> 단어 딕셔너리
int_to_vocab[1]

'||period||'

In [66]:
# 구두점 -> Token 딕셔너리
token_dict

{'.': '||Period||',
 ',': '||Comma||',
 '"': '||Quotation_Mark||',
 ';': '||Semicolon||',
 '!': '||Exclamation_mark||',
 '?': '||Question_mark||',
 '(': '||Left_Parentheses||',
 ')': '||Right_Parentheses||',
 '-': '||Dash||',
 '\n': '||Return||'}

# Check Point

코드스크립트를 재시작할때 이 부분으로부터의 윗부분은 다시 실행할 필요 없게끔

체크포인트 생성

위에서 저장한 pickle에 전처리된 데이터들이 있으므로 그거만 로드해주자

In [67]:
import helper
import problem_unittests as test

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [68]:
def load_preprocess():
    """
    Load the Preprocessed Training data and return them in batches of <batch_size> or less
    """
    return pickle.load(open('preprocess.p', mode='rb'))

---

## 05. Build the Neural Network

RNN 모듈과 forward and backpropagation function을 만들것이다

### 5-0. Check Access to GPU

In [69]:
import torch

train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print("No GPU found. Please use a GPU to train your neural network")

No GPU found. Please use a GPU to train your neural network


### 5-1. Input

이제 우리는 아래와 같은 전처리가 된 '데이터'를 가지고 있다.

* 구두점들 Tokenize 완료 <span style="color:skyblue">ex)</span> `.` $\rightarrow$ `||Period||`
* 모든 Character 소문자로변경
* 단어단위로 split
* 빈도수순으로 넘버링하여 단어 $\rightarrow$ Token(정수)

이제 단순히 892,132개(8십9만2천1백2십3개)의 정수로 이루어진 리스트, 즉 `int_text`

전처리된 텍스트 데이터를 Neural Network가 편하게 받아들일 수 있게 `DataLoader`로 만들어줘야한다

(batch, shuffle, iteration 등을 쉽게 할 수 있음)

#### Batching

우리가 가진 `int_text`, 즉 일렬로 나열된 Tokenize된 단어 리스트

이는 Seinfeld TV 시트콤 스크립트를 단순히 나열한 것이다

우리의 목적인 TV Script Generation을 하러면 

앞의 단어 몇개를 먹였을때, 그 다음 단어로 예측되는 단어를 뱉으면 되는거잖아?

따라서 sequence length개의 단어, $x$ 데이터와 바로 그 다음 단어인 $y$ 데이터를

하나의 Batch로 만들어줄 필요가 있다

<span style="color:skyblue">ex)</span> 주어진 단어리스트가 `words`, sequence length가 `seq_length` 이고

$x$, 즉 입력데이터를 `feature_tensor`, $y$, 즉 정답데이터를 `target_tensor`라고 하면

`words = [1, 2, 3, 4, 5, 6, 7]`

`seq_length = 4`

---

1) `feature_tensor = [1, 2, 3, 4]`
`target_tensor = 5`

2) `feature_tensor = [2, 3, 4, 5]`
`target_tensor = 6`

3) `feature_tensor = [3, 4, 5, 6]`
`target_tensor = 7`

이런식으로 3개의 Batch화 시킬 수 있어야 한다.

#### Tips 1.

어떤 길이 $n$의 리스트 데이터가 있다. 그 데이터를 앞에서부터 $m$개씩 꺼내서 사용(윈도우, 한번 꺼낸 이후에는 다음 순서부터 꺼내기 시작)하면 총 몇번 꺼내서 사용할 수 있을까? 

$\rightarrow n - m + 1$

<span style="color:skyblue">ex)</span> $n$ : [1,2,3,4,5,6,7], $m$ : 3이라면

[1,2,3], [2,3,4], [3,4,5], [4,5,6], [5,6,7] 총 5개

#### Tips 2.

어떤 길이 $n$의 리스트 데이터가 있다. 그 데이터를 앞에서부터 $m$개씩 꺼내서 사용하고 $m+1$번째 데이터를 짝으로써 사용한다면, 총 몇 세트의 데이터를 만들 수 있을까?

$\rightarrow n - m + 1 - 1 = n - m$

<span style="color:skyblue">ex)</span> $n$ : [1,2,3,4,5,6,7], $m$ : 3이라면

* [1,2,3], 4
* [2,3,4], 5
* [3,4,5], 6
* [4,5,6], 7

총 4세트

---

#### Practice



In [75]:
t_text = [1,2,3,4,5,6,7]
seq_length = 3
n_batches = len(t_text) - seq_length
print(n_batches)

4


In [81]:
# (n_batches, seq_length) size의 x데이터, 즉 0 feature_tensors 생성
# n_batches size의 y데이터, 즉 0 target_tensors 생성

feature_tensors = np.zeros( (n_batches, seq_length), dtype=int)
target_tensors = np.zeros( (n_batches), dtype=int)

In [77]:
print(feature_tensors)
print()
print(target_tensors)

[[0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]]

[0 0 0 0]


In [82]:
# 0로만 이루어진 x,y에 데이터를 삽입

for i in range(0, n_batches):
    feature_tensors[i] = t_text[i:i+seq_length]
    target_tensors[i] = t_text[ (i+seq_length) ]

In [79]:
feature_tensors

array([[1, 2, 3],
       [2, 3, 4],
       [3, 4, 5],
       [4, 5, 6]])

In [80]:
target_tensors

array([4, 5, 6, 7])

---

### 5-2. Batching & Make DataLoader function

입력으로 아래의 parameter들을 넣으면 알아서 DataLoader를 반환해주는 함수를 만들자

* `words` : 단어(정수Token)들이 일렬로 나열된 리스트
* `sequence_length` : 하나의 mini batch의 x데이터에 들어갈 단어(Token) 개수
* `batch_size` : 하나의 (큰) batch에 들어갈 mini batch의 개수, sequence의 개수

<span style="color:green">[+]</span> Batch vs mini-batch

1. 일렬로 나열된 단어 리스트 <span style="color:skyblue">ex)</span> [1,2,3,4,5,6,7]
2. 윈도우 크기, 즉 sequence length개의 $x$와 바로 다음의 $y$로 쪼개서 mini-batch만듦
<span style="color:skyblue">ex)</span> [1,2,3] 4 / [2,3,4] 5 / [3,4,5] 6 / [4,5,6] 7
3. 이렇게 만들어진 mini-batch들을 하나의 (큰) Batch에 몇개씩 담느냐가 `batch_size`

In [83]:
from torch.utils.data import TensorDataset, DataLoader

def batch_data(words, sequence_length, batch_size):
    """
    words : The word token(id) of the TV scripts
    sequence_length : The sequence length of 각 (mini)batch
    batch_size : 하나의 batch에 mini batch가 몇개들어가는지, 즉 sequence가 몇개인지
    return : DataLoader with batched data
    """
    
    n_batches = len(words) - sequence_length
    feature_tensors = np.zeros( (n_batches, sequence_length), dtype=int)
    target_tensors = np.zeros( (n_batches), dtype=int)
    
    for i in range(0, n_batches):
        feature_tensors[i] = words[i:i+sequence_length]
        target_tensors[i] = words[ (i+sequence_length) ]
        
    dataset = TensorDataset(torch.from_numpy(feature_tensors),
                            torch.from_numpy(target_tensors)
                           )
    
    dataloader = DataLoader(dataset, shuffle=True, batch_size=batch_size)
    
    return dataloader

### 5-3. Batching & Make DataLoader function Test

0부터 49까지의(총 50개) 리스트를 만들어서 $\leftarrow$ `words`

sequence length = 5, (큰) Batch에 들어가는 mini-batch 개수, 즉 `batch_size`를 10으로 하는 Batch를 뱉는 DataLoader를 만들어보자

In [89]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[ 2,  3,  4,  5,  6],
        [29, 30, 31, 32, 33],
        [24, 25, 26, 27, 28],
        [ 7,  8,  9, 10, 11],
        [17, 18, 19, 20, 21],
        [33, 34, 35, 36, 37],
        [16, 17, 18, 19, 20],
        [35, 36, 37, 38, 39],
        [30, 31, 32, 33, 34],
        [ 5,  6,  7,  8,  9]], dtype=torch.int32)

torch.Size([10])
tensor([ 7, 34, 29, 12, 22, 38, 21, 40, 35, 10], dtype=torch.int32)


<span style="color:red">[!]</span> 

* input 리스트 길이, $n$ : 50
* sequence length, $m$ : 5 이므로

총 45개의 ($x$,$y$) 데이터셋, 즉 mini-batch를 만들 수 있으나

그 45개의 mini-batch중에서 10개씩(`batch_size`)만 묶어서 Batch로 만듦

In [103]:
print(f"mini batch의 개수 : {len(t_loader.dataset)}")
print(f"Batch의 개수 : {len(t_loader)}")

mini batch의 개수 : 45
Batch의 개수 : 5


In [104]:
for d in t_loader:
    print(d)

[tensor([[16, 17, 18, 19, 20],
        [33, 34, 35, 36, 37],
        [43, 44, 45, 46, 47],
        [ 8,  9, 10, 11, 12],
        [41, 42, 43, 44, 45],
        [ 6,  7,  8,  9, 10],
        [24, 25, 26, 27, 28],
        [10, 11, 12, 13, 14],
        [32, 33, 34, 35, 36],
        [ 9, 10, 11, 12, 13]], dtype=torch.int32), tensor([21, 38, 48, 13, 46, 11, 29, 15, 37, 14], dtype=torch.int32)]
[tensor([[38, 39, 40, 41, 42],
        [44, 45, 46, 47, 48],
        [29, 30, 31, 32, 33],
        [28, 29, 30, 31, 32],
        [35, 36, 37, 38, 39],
        [15, 16, 17, 18, 19],
        [19, 20, 21, 22, 23],
        [17, 18, 19, 20, 21],
        [14, 15, 16, 17, 18],
        [12, 13, 14, 15, 16]], dtype=torch.int32), tensor([43, 49, 34, 33, 40, 20, 24, 22, 19, 17], dtype=torch.int32)]
[tensor([[18, 19, 20, 21, 22],
        [20, 21, 22, 23, 24],
        [ 0,  1,  2,  3,  4],
        [22, 23, 24, 25, 26],
        [ 3,  4,  5,  6,  7],
        [36, 37, 38, 39, 40],
        [ 2,  3,  4,  5,  6],
       

### 5-4. Build the Neural Network

PyTorch의 Module class를 통하여 RNN을 구현하자. GRU or LSTM을 사용할 수 있고 
아래와 같은 Function이 구현되어야 한다

* `__init__` : 초기화 함수
* `init_hidden` : LSTM/GRU의 Hidden state에 대한 초기화 함수
* `forward` : Forward propagation 함수

---

`__init__` 함수는 network의 layer를 생성하여 class에 저장해야 한다.

`forward` 함수는 이러한 Layer들을 사용하여 Forward propagation을 실행하고 output, hidden state를 생성한다

이 모델의 output은 모든 sequence가 처리된 후의 (마지막 sequence에 대한) score만 출력한다

In [138]:
import torch.nn as nn

In [182]:
class RNN(nn.Module):
    
    def __init__(self,
                 vocab_size,     # Unique Vocab 개수, 즉 Embedding 직전의 단어당 unit수
                 output_size,    # Unique Vocab 개수, 예측된 단어 Score의 개수
                 embedding_dim,  # Embedding layer로 차원축소시킨 dimension
                 hidden_dim,     # RNN의 cell내 unit수
                 n_layers,
                 dropout=0.5):
        
        super(RNN, self).__init__()
        
        # set class variables
        self.n_layers = n_layers
        self.output_size = output_size
        self.hidden_dim = hidden_dim
        
        # Define model layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # + LSTM(input_size, hidden_dim, n_layers, dropout, batch_first)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, output_size)
        
        
    def forward(self, x, hidden):
        batch_size = x.size(0)      # x : (batch_size, seq_length, input_size) from DataLoader
        x = x.long()                # 정수형 Tensor로 변환
        embeds = self.embedding(x)  # embeds : (batch_size, seq_length, embedding_dim)
        lstm_out, hidden = self.lstm(embeds, hidden)
                                    # lstm_out : (batch_size, seq_length, hidden_dim)
        # Stack-up
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
                                    # lstm_out : (batch_size*seq_length, hidden_dim)
        out = self.dropout(lstm_out)
        out = self.fc(out)          # out : (batch_size*seq_length, output_size)
        out = out.view(batch_size, -1, self.output_size)
                                    # out : (batch_size, seq_length, output_size)
        out = out[:, -1]
                                    # out : (batch_size, 마지막, output_size)
            
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        
        if train_on_gpu:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
            
        return hidden
    

In [183]:
tests.test_rnn(RNN,train_on_gpu)

Tests Passed


### 5-5. Define forward and backpropagation

위에서 만든 RNN 클래스를 이용하여 Forward propagation / Backward propagation를 진행할
함수를 만들자

기능들

* data를 GPU로 move(GPU available일때)
* return : loss, hidden

---

Parameters

* `decoder` : The PyTorch module that holds the NN, 즉 선언한 모델 변수
* `decoder_optimizer` : The PyTorch optimizer for NN
* `criterion` : The PyTorch loss function
* `inp` : NN에 입력될 Input Batch
* `target` : input Batch의 target, 즉 정답


#### Trouble Shooting 


```
RuntimeError: Expected object of scalar type Long but got scalar type int when using CrossEntropyLoss
```

음... 정확히는 모르겠지만 CrossEntropyLoss -> Criterion, 즉 Loss function을 사용할때

criterion(x데이터, y데이터) 이렇게 사용하잖아? 

* `x` : 보통 score? 확률? 따라서 float
* `y` : 정수, LongTensor여야 하나봐



In [295]:
x,y = next(iter(train_loader))
print(f"input의 data type : {x.dtype}")
print(f"target,label의 data type : {y.dtype}")

input의 data type : torch.int32
target,label의 data type : torch.int32


In [297]:
h = rnn.init_hidden(100)
output, h = rnn(x,h)
print(f"output의 data type : {output.dtype}")

output의 data type : torch.float32


---

In [163]:
def forward_back_prop(rnn, optimizer, criterion, inputs, targets, hidden):
    
    # Move data to GPU, if available
    if train_on_gpu:
        inputs, targets = inputs.cuda(), targets.cuda()
        
    # Perform backpropagation and optimization
    hidden = tuple([each.data for each in hidden])
    
    rnn.zero_grad()
    
    output, hidden = rnn(inputs, hidden)
    loss = criterion(output.squeeze(), targets.long())  # target, 즉 label, y 데이터도 LongTensor로 변경시켜주자!
    loss.backward()
    
    nn.utils.clip_grad_norm_(rnn.parameters(), 5)
    optimizer.step()
    
    return loss.item(), hidden

In [144]:
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


---

## 06. Neural Network Training

### 6-1. Train Loop

In [145]:
import time
from time import time

In [168]:
def train_rnn(rnn,
               batch_size,
               optimizer,
               criterion,
               n_epochs,
               show_every_n_batches=100):
    
    batch_losses = []
    
    rnn.train()
    
    print(f"Training for {n_epochs}...")
    for epoch_i in range(1, n_epochs + 1):    # epoch개수 (1부터)
        start = time()
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        # batch_i : 몇번째 batch인지 (1부터)
        # inputs, labels : DataLoader로 받은 x,y
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # 꽉찬 Batch로만 Training
            n_batches = len(train_loader.dataset) // batch_size
            if(batch_i > n_batches):
                break
                
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden )
            
            # 매 Batch마다의 loss값을 list에 쌓아두기
            batch_losses.append(loss)
            
            # Loss stats 출력
            if batch_i % show_every_n_batches == 0:
                print(f"Epoch: {epoch_i}/{n_epochs}  Loss: {np.average(batch_losses)}")
                # batch_losses 초기화
                batch_losses = []
                
        end = time()
        print(f"epoch time spend: {end-start}[sec]")
        
    return rnn
        

### 6-2. Hyperparameters

* `sequence_length` : Sequence length (RNN으로 누적해서 몇개의 단어까지 볼건지)
* `batch_size` : 하나의 Batch에 몇개의 x,y 데이터셋을 넣을건지
* `num_epochs` : Training epoch를 몇번할건지
* `learning_rate` : Learning rate
* `vocab_size` : Unique vocabulary 개수 (또는 모델의 input layer's unit수, 임베딩전)
* `output_size` : = `vocab_size` (우리는 다음 단어를 예측하는 것이므로)
* `embedding_dim` : Embedding layer로 몇차원까지 축소할지
* `hidden_dim` : RNN cell내의 unit수
* `n_layers` : RNN layer 개수
* `show_every_n_batches` : 몇번의 Batch training 마다 loss(평균)을 출력할건지


#### Data parameters

In [187]:
sequence_length = 3  # of words in a sequence
batch_size = 100

train_loader = batch_data(int_text, sequence_length, batch_size)

#### Training parameters

In [188]:
num_epochs = 1
learning_rate = 0.01

#### Model parameters

In [189]:
vocab_size = len(vocab_to_int)     # = 21,383
output_size = vocab_size
embedding_dim = 400
hidden_dim = 512
n_layers = 2

show_every_n_batches = 500

### 6-3. Train

<span style="color:red">[!]</span> GPU가 없으므로 간단하게 epoch는 2로만 해보자

In [154]:
import workspace_utils
from workspace_utils import active_session

In [190]:
rnn = RNN(vocab_size,
          output_size,
          embedding_dim,
          hidden_dim,
          n_layers,
          dropout=0.5)

In [192]:
if train_on_gpu:
    rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# with active_session():
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)
    
    
# Save the trained model
helper.save_model('./save/trained_rnn_epoch1', trained_rnn)
print("Model Trained and Saved")

Training for 1...
Epoch: 1/1  Loss: 5.48026679611206
Epoch: 1/1  Loss: 5.350816349983216
Epoch: 1/1  Loss: 5.25140052318573
Epoch: 1/1  Loss: 5.262663858413696
Epoch: 1/1  Loss: 5.202031468391419
Epoch: 1/1  Loss: 5.191882739067077
Epoch: 1/1  Loss: 5.1273360452651975
Epoch: 1/1  Loss: 5.129231917381286
Epoch: 1/1  Loss: 5.112112811088562
Epoch: 1/1  Loss: 5.089050556659698
Epoch: 1/1  Loss: 5.1027843661308285
Epoch: 1/1  Loss: 5.067692330360413
Epoch: 1/1  Loss: 5.080345338344574
Epoch: 1/1  Loss: 5.100670188903808
Epoch: 1/1  Loss: 5.077177362442017
Epoch: 1/1  Loss: 5.061386499404907
Epoch: 1/1  Loss: 5.108796964645386
epoch time spend: 5265.764845371246[sec]
Model Trained and Saved


<span style="color:red">[!]</span> model save error 관련 practice

In [177]:
def save_model(filename, decoder):
    save_filename = os.path.splitext(os.path.basename(filename))[0] + '.pt'
    torch.save(decoder, save_filename)

In [185]:
rnn

RNN(
  (embedding): Embedding(21383, 400)
  (lstm): LSTM(400, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=21383, bias=True)
)

In [186]:
torch.save(rnn, 'save_test.pt')

---

## Checkpoint

In [195]:
import torch
import helper
import problem_unittests as test

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn_epoch1')

In [ ]:
# helper.py의 load_model 함수
# 단순히 입력된 filename에다가 .pt를 붙여서 불러옴
def load_model(filename):
    save_filename = os.path.splitext(os.path.basename(filename))[0] + '.pt'
    return torch.load(save_filename)

---

## 07. Generate TV Script

저장된 훈련완료 network를 통해서 새로운 ~~'가짜'~~ Seinfeld TV script를 만들어보자

'텍스트'를 생성하려면 network는 하나의 단어로 시작하고 사용자가 임의로 정해준 (예측)길이까지 예측을 반복한다

* `prime_id` : 시작 단어 (integer)
* `predict_len` : 예측하여 생성하고 싶은 단어 개수

In [298]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)   # (size)의 array를 pad_value로 채운다
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, axis=1) # 축기준으로 왼쪽(-1)으로 요소들 이동
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

---

## 08. Generate a New Script

* `gen_length` : 새롭게 생성할 Script의 단어개수
* `prime_word` : 처음에 넣어줄 prime 단어 `jerry`, `elaine`, `george` 등 추천

In [299]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

jerry: you,. you, i just got...

elaine: you know.

george: i know what?

george: yeah, no i just just, you?

elaine: you........

jerry: i know, i got you?

kramer: you. i don't know. you?

kramer: no.

jerry: i got a.

kramer: no.

elaine: yeah.

george: yeah. i don't want to be..

george:(, the the

jerry: i got the

jerry:(,(.. you you, i got a.

elaine: i just got.

kramer: oh, i just just just, you.

elaine: oh, i don't know, i know.

elaine: i got you, i don't have a. i don't want me.

jerry:(the?

george: i have you, the. you, i don't want me to a-.

george: oh yeah, i just was the

kramer:(,.

jerry:(the?

elaine:(, you, you have to the, and, i don't want you know, i got you, you know what is it.....

george:(the.

kramer: oh!

jerry: yeah, i have a, and.

kramer: i know, you have to you?

george:(,.

jerry: i got a.

elaine: i have to be, i got you.

jerry: oh, i have a- the

jerry: i don't know, the, you don't think i don't want to you, you don't think it's...

jerry: i got.